## Import Library

In [245]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam


## Load Dataset

In [246]:
sleep_df = pd.read_csv(
    "/kaggle/input/fitbit/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/sleepDay_merged.csv",
    parse_dates=["SleepDay"],
    date_format="%m/%d/%Y %I:%M:%S %p"
)


steps_df = pd.read_csv(
    "/kaggle/input/fitbit/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/hourlySteps_merged.csv",
    parse_dates=["ActivityHour"],
     date_format="%m/%d/%Y %I:%M:%S %p"
)

cal_df = pd.read_csv(
    "/kaggle/input/fitbit/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/hourlyCalories_merged.csv",
    parse_dates=["ActivityHour"],
    date_format="%m/%d/%Y %I:%M:%S %p"
)

hr_df = pd.read_csv(
    "/kaggle/input/fitbit/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/heartrate_seconds_merged.csv",
    parse_dates=["Time"],
    date_format="%m/%d/%Y %I:%M:%S %p"
)


In [247]:
sleep_df = sleep_df.sort_values(["Id", "SleepDay"])
steps_df = steps_df.sort_values(["Id", "ActivityHour"])
cal_df = cal_df.sort_values(["Id", "ActivityHour"])
hr_df = hr_df.sort_values(["Id", "Time"])

# Ensure datetime (extra safety)
hr_df["Time"] = pd.to_datetime(hr_df["Time"])

## RESAMPLE HEART RATE → HOURLY

In [248]:
hr_hourly = (
    hr_df
    .set_index("Time")
    .groupby("Id")["Value"]
    .resample("h")        # ✅ lowercase h (future-proof)
    .mean()
    .reset_index()
    .rename(columns={"Value": "heart_rate"})
)

## MERGE HOURLY

In [249]:
df = steps_df.merge(cal_df, on=["Id", "ActivityHour"])
df = df.merge(hr_hourly, left_on=["Id", "ActivityHour"], right_on=["Id", "Time"])
df.drop(columns=["Time"], inplace=True)

## MERGE SLEEP (DAILY → HOURLY)

In [250]:
sleep_df["SleepDate"] = sleep_df["SleepDay"].dt.date
df["SleepDate"] = df["ActivityHour"].dt.date

df = df.merge(
    sleep_df[["Id", "SleepDate", "TotalMinutesAsleep"]],
    on=["Id", "SleepDate"],
    how="inner"
)

## FEATURE ENGINEERING (STRESS, RHR)

In [251]:
rhr = (
    df.groupby("Id")["heart_rate"]
    .quantile(0.25)
    .reset_index()
    .rename(columns={"heart_rate": "resting_hr"})
)

df = df.merge(rhr, on="Id")
df["stress"] = df["heart_rate"] - df["resting_hr"]
df = df.dropna()


## PSEUDO TARGETS (5 TASK)

In [252]:
df["sleep_score"] = np.clip((df["TotalMinutesAsleep"] / 480) * 100, 0, 100)
df["rhr_score"] = np.clip(100 - (df["resting_hr"] - 60) * 2, 0, 100)
df["hrv_score"] = np.clip(100 - abs(df["heart_rate"] - df["resting_hr"]), 0, 100)
df["recovery_score"] = np.clip(100 - df["stress"] * 5, 0, 100)

df["readiness_score"] = (
    0.30 * df["sleep_score"] +
    0.25 * df["recovery_score"] +
    0.20 * df["hrv_score"] +
    0.15 * df["rhr_score"]
)

## INPUT & NORMALIZATION

In [253]:
FEATURES = ["StepTotal", "Calories", "heart_rate", "stress"]
TARGETS = ["sleep_score", "hrv_score", "rhr_score", "recovery_score", "readiness_score"]

scaler = MinMaxScaler()
df[FEATURES] = scaler.fit_transform(df[FEATURES])

## TIME SERIES WINDOW (PER USER)

In [254]:
SEQ_LEN = 6
X, y = [], []

for user in df["Id"].unique():
    user_df = df[df["Id"] == user].sort_values("ActivityHour")

    for i in range(len(user_df) - SEQ_LEN):
        X.append(user_df[FEATURES].iloc[i:i+SEQ_LEN].values)
        y.append(user_df[TARGETS].iloc[i+SEQ_LEN].values)

X = np.array(X)
y = np.array(y)

## MODEL (PRETRAIN)

In [256]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# =========================
# MODEL
# =========================

input_layer = Input(shape=(SEQ_LEN, len(FEATURES)))

shared = LSTM(64, return_sequences=True, name="encoder_lstm_1")(input_layer)
shared = LSTM(32, name="encoder_lstm_2")(shared)
shared = Dropout(0.3)(shared)

sleep_out = Dense(1, name="sleep")(shared)
hrv_out = Dense(1, name="hrv")(shared)
rhr_out = Dense(1, name="rhr")(shared)
recovery_out = Dense(1, name="recovery")(shared)
readiness_out = Dense(1, name="readiness")(shared)

fitbit_model = Model(
    input_layer,
    [sleep_out, hrv_out, rhr_out, recovery_out, readiness_out]
)

# =========================
# COMPILE (BENAR)
# =========================

fitbit_model.compile(
    optimizer=Adam(0.001),
    loss={
        "sleep": "mse",
        "hrv": "mse",
        "rhr": "mse",
        "recovery": "mse",
        "readiness": "mse"
    },
    metrics={
        "sleep": "mae",
        "hrv": "mae",
        "rhr": "mae",
        "recovery": "mae",
        "readiness": "mae"
    }
)

# =========================
# FIT
# =========================

fitbit_model.fit(
    X,
    {
        "sleep": y[:, 0],
        "hrv": y[:, 1],
        "rhr": y[:, 2],
        "recovery": y[:, 3],
        "readiness": y[:, 4],
    },
    epochs=50,
    batch_size=32,
    validation_split=0.2
)


Epoch 1/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - hrv_loss: 7760.1440 - hrv_mae: 87.4185 - loss: 33752.6641 - readiness_loss: 5282.8208 - readiness_mae: 71.4404 - recovery_loss: 4746.8599 - recovery_mae: 58.0561 - rhr_loss: 8346.9639 - rhr_mae: 90.8836 - sleep_loss: 7615.8740 - sleep_mae: 85.3037 - val_hrv_loss: 6726.1318 - val_hrv_mae: 81.7315 - val_loss: 29439.7109 - val_readiness_loss: 4649.6714 - val_readiness_mae: 67.4627 - val_recovery_loss: 3984.6064 - val_recovery_mae: 54.1916 - val_rhr_loss: 6883.0669 - val_rhr_mae: 82.6840 - val_sleep_loss: 6973.7441 - val_sleep_mae: 81.7630
Epoch 2/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - hrv_loss: 6550.3970 - hrv_mae: 80.1943 - loss: 29052.8379 - readiness_loss: 4585.8887 - readiness_mae: 66.3712 - recovery_loss: 4055.2173 - recovery_mae: 53.6217 - rhr_loss: 7183.9326 - rhr_mae: 84.2675 - sleep_loss: 6677.4038 - sleep_mae: 79.6482 - val_hrv_loss: 6151.1206 - val_hrv_mae: 78.1225 - val_loss: 26874.0918 - val_readiness_loss: 4181.1909 -

## SIMPAN ENCODER

In [257]:
encoder = Model(
    inputs=fitbit_model.input,
    outputs=fitbit_model.get_layer("encoder_lstm_2").output
)

encoder.save("fitbit_encoder2.keras")

## Fine Tuning

In [258]:
df = pd.read_csv(
    "/kaggle/input/smartwatch-clean-dataset/smartwatch_clean.csv"
)

In [259]:
df.head()

,heart_rate,blood_oxygen_level,step_count,sleep_duration,activity_level,stress_level
0,58.939776,98.809650,5450.390578,7.167236,Highly Active,1
1,74.883189,98.532195,727.601610,6.538239,Highly Active,5
2,247.803052,97.052954,2826.521994,6.515466,Highly Active,5
3,40.000000,96.894213,13797.338044,7.367790,Active,3
4,61.950165,98.583797,15679.067648,6.515466,Highly Active,6


In [260]:
from sklearn.preprocessing import LabelEncoder

dataolahencoding = df.copy() # Membuat salinan dataset untuk mencegah modifikasi dataset asli

# Label encoding 
encoder = LabelEncoder()
dataolahencoding['heart_rate_encod'] = encoder.fit_transform(dataolahencoding['heart_rate'])

In [261]:
dataolahencoding = dataolahencoding.drop(['heart_rate'], axis=1)
dataolahencoding.head(100)

,heart_rate_encod,blood_oxygen_level_encod,step_count_encod,sleep_duration_encod,activity_level_encod,stress_level_encod
0,1240,6185,5207,6330,1,0
1,4508,5703,947,4802,1,5
2,9299,3043,3220,4743,1,5
3,0,2772,8356,6747,0,3
4,1711,5789,8692,4743,1,6
...,...,...,...,...,...,...
95,0,707,7660,8342,0,4
96,7987,4604,116,5353,0,6
97,96,8012,3120,5408,0,6
98,9154,7804,4945,4903,2,1


In [271]:
# =========================
# PERBAIKAN 1: Gunakan data asli, bukan yang sudah di-label-encode
# =========================

# 1. Gunakan data asli dari smartwatch
data_numeric = df.copy()

# 2. Cek tipe data semua kolom
print("=== Data Types ===")
print(data_numeric.dtypes)
print("\n=== Sample Data ===")
print(data_numeric.head())
print("\n=== Unique values in stress_level ===")
print(data_numeric["stress_level"].unique())

# 3. Konversi kolom 'stress_level' ke numerik jika masih string
if data_numeric["stress_level"].dtype == 'object':
    try:
        # Coba konversi langsung ke numeric
        data_numeric["stress_level"] = pd.to_numeric(data_numeric["stress_level"], errors='coerce')
        print("stress_level converted to numeric")
    except:
        # Jika mengandung nilai non-numeric, lakukan mapping
        stress_mapping = {
            'Low': 1,
            'Medium': 3,
            'High': 5,
            'Very High': 7,
            'Extreme': 9
        }
        # Atau jika sudah dalam format angka sebagai string
        data_numeric["stress_level"] = data_numeric["stress_level"].astype(str).str.extract('(\d+)').astype(float)
        print("stress_level extracted numeric values")

# 4. Konversi kolom lain yang mungkin string
for col in ["heart_rate", "blood_oxygen_level", "step_count", "sleep_duration"]:
    if data_numeric[col].dtype == 'object':
        data_numeric[col] = pd.to_numeric(data_numeric[col], errors='coerce')
        print(f"{col} converted to numeric")

# 5. Handle missing values jika ada
print(f"\n=== Missing values ===")
print(data_numeric.isnull().sum())

# Drop atau impute missing values
data_numeric = data_numeric.dropna()
print(f"Data shape after dropping NA: {data_numeric.shape}")

# 6. One-hot encoding untuk kolom kategorikal 'activity_level'
if 'activity_level' in data_numeric.columns:
    # Cek nilai unik
    print(f"\n=== Unique activity_level values ===")
    print(data_numeric["activity_level"].unique())
    
    # One-hot encoding
    data_numeric = pd.get_dummies(data_numeric, columns=['activity_level'], prefix='activity')

# 7. Hitung scores dengan nilai asli (bukan encoded)
resting_hr = data_numeric['heart_rate'].quantile(0.25)
print(f"\nResting heart rate (25th percentile): {resting_hr}")

data_numeric["sleep_score"] = np.clip((data_numeric["sleep_duration"] / 8) * 100, 0, 100)
data_numeric["rhr_score"] = np.clip(100 - (data_numeric["heart_rate"] - resting_hr), 0, 100)
data_numeric["hrv_score"] = np.clip(100 - abs(data_numeric["heart_rate"] - data_numeric["heart_rate"].mean()), 0, 100)
data_numeric["recovery_score"] = np.clip(100 - data_numeric["stress_level"] * 10, 0, 100)

data_numeric["readiness_score"] = (
    0.30 * data_numeric["sleep_score"] +
    0.25 * data_numeric["recovery_score"] +
    0.20 * data_numeric["hrv_score"] +
    0.15 * data_numeric["rhr_score"]
)

# 8. Cek hasil perhitungan
print("\n=== Score Statistics ===")
print(data_numeric[["sleep_score", "rhr_score", "hrv_score", "recovery_score", "readiness_score"]].describe())

# 9. Pilih fitur dan normalisasi
FEATURES = ["heart_rate", "blood_oxygen_level", "step_count", "stress_level"]
# Tambahkan kolom activity jika ada
activity_cols = [col for col in data_numeric.columns if col.startswith('activity_')]
FEATURES.extend(activity_cols)

print(f"\nFeatures to use: {FEATURES}")

TARGETS = ["sleep_score", "hrv_score", "rhr_score", "recovery_score", "readiness_score"]

# 10. Normalisasi fitur
scaler = MinMaxScaler()
data_numeric[FEATURES] = scaler.fit_transform(data_numeric[FEATURES])

# 11. Siapkan data time series
SEQ_LEN = 6
X, y = [], []

for i in range(len(data_numeric) - SEQ_LEN):
    X.append(data_numeric[FEATURES].iloc[i:i+SEQ_LEN].values)
    y.append(data_numeric[TARGETS].iloc[i+SEQ_LEN].values)

X = np.array(X)
y = np.array(y)

print(f"\n=== Data Shapes ===")
print(f"Shape X: {X.shape}")  # Harus (samples, 6, num_features)
print(f"Shape y: {y.shape}")  # Harus (samples, 5)

# Lanjutkan dengan bagian encoder dan model...

=== Data Types ===
heart_rate            float64
blood_oxygen_level    float64
step_count            float64
sleep_duration        float64
activity_level         object
stress_level           object
dtype: object

=== Sample Data ===
   heart_rate  blood_oxygen_level    step_count  sleep_duration  \
0   58.939776           98.809650   5450.390578        7.167236   
1   74.883189           98.532195    727.601610        6.538239   
2  247.803052           97.052954   2826.521994        6.515466   
3   40.000000           96.894213  13797.338044        7.367790   
4   61.950165           98.583797  15679.067648        6.515466   

  activity_level stress_level  
0  Highly Active            1  
1  Highly Active            5  
2  Highly Active            5  
3         Active            3  
4  Highly Active            6  

=== Unique values in stress_level ===
['1' '5' '3' '6' '10' '2' '8' '9' '4' 'Very High' '7']
stress_level converted to numeric

=== Missing values ===
heart_rate         

In [281]:
# =========================
# 1. MEMBUAT DATA DUMMY
# =========================
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# =========================
# 1. BUAT DATA DUMMY YANG SESUAI DENGAN MODEL ASLI
# =========================
# Model asli menggunakan 4 fitur:
# ["StepTotal", "Calories", "heart_rate", "stress"]

dummy_data_correct = {
    "StepTotal": [1000, 1500, 2000, 500, 3000, 1000],  # mirip dengan step_count
    "Calories": [50, 75, 100, 25, 150, 50],  # kalori yang dibakar
    "heart_rate": [70, 72, 68, 75, 65, 71],  # BPM
    "stress": [3, 4, 2, 5, 1, 3]  # level stress
}

# Konversi ke DataFrame
dummy_df = pd.DataFrame(dummy_data_correct)

# Gunakan FEATURES yang sama dengan model asli
FEATURES_ORIGINAL = ["StepTotal", "Calories", "heart_rate", "stress"]

# Normalisasi (gunakan scaler asli jika ada)
# Untuk demo, kita buat scaler baru
scaler_original = MinMaxScaler()
dummy_scaled = scaler_original.fit_transform(dummy_df[FEATURES_ORIGINAL])

# Reshape untuk model: (1 sampel, 6 timesteps, 4 fitur)
dummy_sequence = dummy_scaled.reshape(1, 6, 4)

print(f"Shape data dummy yang benar: {dummy_sequence.shape}")  # Harus (1, 6, 4)

# =========================
# 2. LOAD MODEL YANG SUDAH DILATIH
# =========================
from tensorflow.keras.models import load_model

# Load model yang sudah disimpan (ganti dengan path model Anda)
# fitbit_model.save("fitbit_model.h5")  # Jika Anda menyimpan model lengkap
# encoder.save("encoder.h5")  # Jika Anda menyimpan encoder

# Untuk contoh ini, kita akan gunakan model yang sudah ada di memori
# Jika Anda sudah menyimpan model, gunakan:
# model = load_model("fitbit_model.h5")

# Atau gunakan model yang sudah dilatih di notebook
model = fitbit_model  # Model dari training sebelumnya

# =========================
# 3. MELAKUKAN PREDIKSI
# =========================
print("\n" + "="*50)
print("MELAKUKAN PREDIKSI UNTUK DATA DUMMY")
print("="*50)

# Prediksi dengan model
predictions = model.predict(dummy_sequence)

# Prediksi akan mengembalikan 5 output berbeda (untuk 5 target)
sleep_pred, hrv_pred, rhr_pred, recovery_pred, readiness_pred = predictions

# =========================
# 4. MENAMPILKAN HASIL PREDIKSI
# =========================
print("\nHASIL PREDIKSI:")
print("-" * 30)
print(f"Sleep Score: {sleep_pred[0][0]:.2f} / 100")
print(f"HRV Score: {hrv_pred[0][0]:.2f} / 100")
print(f"RHR Score: {rhr_pred[0][0]:.2f} / 100")
print(f"Recovery Score: {recovery_pred[0][0]:.2f} / 100")
print(f"Readiness Score: {readiness_pred[0][0]:.2f} / 100")

# =========================
# 5. INTERPRETASI HASIL
# =========================
print("\n" + "="*50)
print("INTERPRETASI HASIL")
print("="*50)

def interpret_score(score, score_name):
    """Fungsi untuk menginterpretasikan skor"""
    if score >= 80:
        status = "Sangat Baik"
    elif score >= 60:
        status = "Baik"
    elif score >= 40:
        status = "Cukup"
    elif score >= 20:
        status = "Kurang"
    else:
        status = "Buruk"
    
    return f"{score_name}: {score:.1f} ({status})"

print("\nStatus Kesehatan Berdasarkan Skor:")
print("-" * 40)
print(interpret_score(sleep_pred[0][0], "Kualitas Tidur"))
print(interpret_score(hrv_pred[0][0], "Variabilitas Denyut Jantung"))
print(interpret_score(rhr_pred[0][0], "Denyut Jantung Istirahat"))
print(interpret_score(recovery_pred[0][0], "Pemulihan"))
print(interpret_score(readiness_pred[0][0], "Kesiapan Harian"))

# =========================
# 6. REKOMENDASI BERDASARKAN HASIL
# =========================
print("\n" + "="*50)
print("REKOMENDASI")
print("="*50)

readiness = readiness_pred[0][0]
sleep = sleep_pred[0][0]
recovery = recovery_pred[0][0]

if readiness >= 70:
    print("✅ KESIMPULAN: Anda dalam kondisi optimal untuk aktivitas berat!")
    print("   Rekomendasi:")
    print("   - Lanjutkan rutinitas olahraga Anda")
    print("   - Pertahankan pola tidur yang baik")
elif readiness >= 50:
    print("⚠️ KESIMPULAN: Kondisi Anda cukup baik untuk aktivitas ringan-sedang")
    print("   Rekomendasi:")
    print("   - Lakukan olahraga ringan")
    print("   - Perhatikan kualitas tidur")
    if sleep < 60:
        print("   - Prioritaskan tidur yang cukup malam ini")
else:
    print("❌ KESIMPULAN: Anda butuh lebih banyak istirahat")
    print("   Rekomendasi:")
    print("   - Kurangi intensitas olahraga")
    print("   - Tingkatkan waktu tidur")
    print("   - Pertimbangkan teknik relaksasi")

Shape data dummy yang benar: (1, 6, 4)

MELAKUKAN PREDIKSI UNTUK DATA DUMMY
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step

HASIL PREDIKSI:
------------------------------
Sleep Score: 55.50 / 100
HRV Score: 57.80 / 100
RHR Score: 57.93 / 100
Recovery Score: 48.17 / 100
Readiness Score: 52.00 / 100

INTERPRETASI HASIL

Status Kesehatan Berdasarkan Skor:
----------------------------------------
Kualitas Tidur: 55.5 (Cukup)
Variabilitas Denyut Jantung: 57.8 (Cukup)
Denyut Jantung Istirahat: 57.9 (Cukup)
Pemulihan: 48.2 (Cukup)
Kesiapan Harian: 52.0 (Cukup)

REKOMENDASI
⚠️ KESIMPULAN: Kondisi Anda cukup baik untuk aktivitas ringan-sedang
   Rekomendasi:
   - Lakukan olahraga ringan
   - Perhatikan kualitas tidur
   - Prioritaskan tidur yang cukup malam ini


## Save Model

In [282]:
# Cek apakah fitbit_model ada
print("Apakah fitbit_model ada di memory?:", 'fitbit_model' in locals() or 'fitbit_model' in globals())

# Cek arsitektur model
if 'fitbit_model' in locals():
    print("\n=== INFO MODEL FITBIT ===")
    print(f"Nama: {fitbit_model.name}")
    print(f"Jumlah layer: {len(fitbit_model.layers)}")
    print(f"Input shape: {fitbit_model.input_shape}")
    print(f"Output shape: {fitbit_model.output_shape}")
    fitbit_model.summary()

Apakah fitbit_model ada di memory?: True

=== INFO MODEL FITBIT ===
Nama: functional_34
Jumlah layer: 9
Input shape: (None, 6, 4)
Output shape: [(None, 1), (None, 1), (None, 1), (None, 1), (None, 1)]


Model: "functional_34"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_22      │ (None, 6, 4)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_lstm_1      │ (None, 6, 64)     │     17,664 │ input_layer_22[0… │
│ (LSTM)              │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_lstm_2      │ (None, 32)        │     12,416 │ encoder_lstm_1[0… │
│ (LSTM)              │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_17          │ (None, 32)        │          0 │ encoder_lstm_2[0… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sleep (Dense)       │ (None, 1)         │         33 │ dropout_17[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hrv (Dense)         │ (None, 1)         │         33 │ dropout_17[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rhr (Dense)         │ (None, 1)         │         33 │ dropout_17[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ recovery (Dense)    │ (None, 1)         │         33 │ dropout_17[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ readiness (Dense)   │ (None, 1)         │         33 │ dropout_17[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 90,737 (354.45 KB)

 Trainable params: 30,245 (118.14 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 60,492 (236.30 KB)

In [284]:
# =========================
# SIMPAN MODEL LENGKAP
# =========================

# 1. Simpan model Keras format .keras (rekomendasi)
fitbit_model.save("fitbit_complete_model.keras")
print("✅ Model lengkap disimpan sebagai 'fitbit_complete_model.keras'")

# 2. Simpan juga dalam format .h5 (compatibility)
fitbit_model.save("fitbit_complete_model.h5")
print("✅ Model lengkap disimpan sebagai 'fitbit_complete_model.h5'")


✅ Model lengkap disimpan sebagai 'fitbit_complete_model.keras'
✅ Model lengkap disimpan sebagai 'fitbit_complete_model.h5'


In [285]:
import joblib
import json
import pickle

# 1. Simpan scaler yang digunakan
joblib.dump(scaler, 'fitbit_scaler.pkl')
print("✅ Scaler disimpan sebagai 'fitbit_scaler.pkl'")

# 2. Simpan informasi fitur dan target
model_info = {
    "FEATURES": FEATURES,
    "TARGETS": TARGETS,
    "SEQ_LEN": SEQ_LEN,
    "input_shape": fitbit_model.input_shape,
    "output_shape": fitbit_model.output_shape,
    "training_date": pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S"),
    "epochs_trained": 20  # Sesuaikan dengan jumlah epoch yang benar
}

with open('model_info.json', 'w') as f:
    json.dump(model_info, f, indent=2)
print("✅ Model info disimpan sebagai 'model_info.json'")

# 3. Simpan dalam satu paket
import dill
model_package = {
    "model": fitbit_model,
    "scaler": scaler,
    "features": FEATURES,
    "targets": TARGETS,
    "seq_len": SEQ_LEN
}

with open('fitbit_model_package.pkl', 'wb') as f:
    dill.dump(model_package, f)
print("✅ Model package disimpan sebagai 'fitbit_model_package.pkl'")

✅ Scaler disimpan sebagai 'fitbit_scaler.pkl'
✅ Model info disimpan sebagai 'model_info.json'
✅ Model package disimpan sebagai 'fitbit_model_package.pkl'


In [286]:
# =========================
# VERIFIKASI: LOAD MODEL YANG SUDAH DISIMPAN
# =========================

from tensorflow.keras.models import load_model

print("\n" + "="*50)
print("VERIFIKASI LOAD MODEL")
print("="*50)

# Load model dari file
loaded_model = load_model("fitbit_complete_model.keras")
print("✅ Model berhasil di-load dari file")

# Cek apakah model sama
print(f"\nNama model loaded: {loaded_model.name}")
print(f"Input shape loaded: {loaded_model.input_shape}")
print(f"Output shape loaded: {loaded_model.output_shape}")

# Test prediksi dengan data dummy
test_input = np.random.randn(1, 6, 4)  # (samples, timesteps, features)
pred_original = fitbit_model.predict(test_input)
pred_loaded = loaded_model.predict(test_input)

# Bandingkan hasil
print(f"\nPerbandingan prediksi:")
print(f"Original model - Sleep score: {pred_original[0][0][0]:.6f}")
print(f"Loaded model   - Sleep score: {pred_loaded[0][0][0]:.6f}")
print(f"Perbedaan: {abs(pred_original[0][0][0] - pred_loaded[0][0][0]):.10f}")

if np.allclose(pred_original[0][0][0], pred_loaded[0][0][0], rtol=1e-5):
    print("✅ Model loaded identik dengan model original!")
else:
    print("⚠️ Ada perbedaan kecil antara model original dan loaded")


VERIFIKASI LOAD MODEL
✅ Model berhasil di-load dari file

Nama model loaded: functional_34
Input shape loaded: (None, 6, 4)
Output shape loaded: [(None, 1), (None, 1), (None, 1), (None, 1), (None, 1)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step

Perbandingan prediksi:
Original model - Sleep score: 55.498810
Loaded model   - Sleep score: 55.498810
Perbedaan: 0.0000000000
✅ Model loaded identik dengan model original!


In [ ]:
# =========================
# SCRIPT LENGKAP: TEST, SIMPAN, DAN VERIFIKASI
# =========================

import numpy as np
import pandas as pd
import joblib
import json
from tensorflow.keras.models import load_model, save_model
from sklearn.preprocessing import MinMaxScaler

print("="*50)
print("SISTEM PENYIMPANAN MODEL")
print("="*50)

# 1. Pastikan model ada
if 'fitbit_model' not in locals() and 'fitbit_model' not in globals():
    print("❌ ERROR: Model 'fitbit_model' tidak ditemukan di memory!")
    print("   Pastikan Anda sudah menjalankan training sebelumnya.")
else:
    print("✅ Model 'fitbit_model' ditemukan di memory")
    
    # 2. Buat data dummy untuk testing
    print("\n1. Membuat data dummy untuk testing...")
    dummy_data = np.array([
        [0.1, 0.2, 0.3, 0.4],
        [0.2, 0.3, 0.4, 0.5],
        [0.3, 0.4, 0.5, 0.6],
        [0.4, 0.5, 0.6, 0.7],
        [0.5, 0.6, 0.7, 0.8],
        [0.6, 0.7, 0.8, 0.9]
    ]).reshape(1, 6, 4)
    
    # 3. Test prediksi
    print("2. Testing prediksi dengan model...")
    predictions = fitbit_model.predict(dummy_data)
    sleep_pred = predictions[0][0][0]
    print(f"   Hasil prediksi sleep_score: {sleep_pred:.4f}")
    
    # 4. Simpan model
    print("\n3. Menyimpan model...")
    
    # Format .keras (rekomendasi TensorFlow 2.x)
    fitbit_model.save("fitbit_full_model.keras")
    print("   ✅ Model disimpan: 'fitbit_full_model.keras'")
    
    # Format .h5 (untuk compatibility)
    fitbit_model.save("fitbit_full_model.h5")
    print("   ✅ Model disimpan: 'fitbit_full_model.h5'")
    
    # 5. Simpan metadata
    print("\n4. Menyimpan metadata...")
    
    # Info model
    model_metadata = {
        "model_name": "Fitbit Multi-Task LSTM",
        "version": "1.0",
        "input_features": FEATURES,
        "output_targets": TARGETS,
        "sequence_length": SEQ_LEN,
        "input_shape": str(fitbit_model.input_shape),
        "output_shapes": [str(out.shape) for out in fitbit_model.output_shape],
        "date_saved": pd.Timestamp.now().isoformat()
    }
    
    with open('model_metadata.json', 'w') as f:
        json.dump(model_metadata, f, indent=2)
    print("   ✅ Metadata disimpan: 'model_metadata.json'")
    
    # 6. Verifikasi load
    print("\n5. Verifikasi load model...")
    try:
        loaded_model = load_model("fitbit_full_model.keras")
        loaded_predictions = loaded_model.predict(dummy_data)
        loaded_sleep = loaded_predictions[0][0][0]
        
        print(f"   Original prediction: {sleep_pred:.6f}")
        print(f"   Loaded prediction:   {loaded_sleep:.6f}")
        
        if abs(sleep_pred - loaded_sleep) < 0.0001:
            print("   ✅ Model berhasil di-load dan memberikan hasil yang sama!")
        else:
            print("   ⚠️ Ada perbedaan kecil, tapi model berhasil di-load")
            
    except Exception as e:
        print(f"   ❌ Error loading model: {e}")
    
    # 7. Buat contoh penggunaan
    print("\n" + "="*50)
    print("CONTOH PENGGUNAAN MODEL")
    print("="*50)
    
    example_code = '''
# ====================================
# CARA MENGGUNAKAN MODEL YANG SUDAH DISIMPAN
# ====================================

# 1. Load model
from tensorflow.keras.models import load_model
model = load_model("fitbit_full_model.keras")

# 2. Load metadata
import json
with open('model_metadata.json', 'r') as f:
    metadata = json.load(f)

FEATURES = metadata["input_features"]
TARGETS = metadata["output_targets"]
SEQ_LEN = metadata["sequence_length"]

# 3. Buat data input (contoh: 6 jam terakhir)
# Data harus sudah dinormalisasi dan memiliki shape (1, 6, 4)
import numpy as np

# Contoh data 6 jam terakhir
data_6jam = np.array([
    [0.1, 0.2, 0.3, 0.4],  # jam t-5
    [0.2, 0.3, 0.4, 0.5],  # jam t-4
    [0.3, 0.4, 0.5, 0.6],  # jam t-3
    [0.4, 0.5, 0.6, 0.7],  # jam t-2
    [0.5, 0.6, 0.7, 0.8],  # jam t-1
    [0.6, 0.7, 0.8, 0.9]   # jam t
]).reshape(1, SEQ_LEN, len(FEATURES))

# 4. Lakukan prediksi
predictions = model.predict(data_6jam)

# 5. Ekstrak hasil
sleep_score = predictions[0][0][0] * 100  # konversi ke skala 0-100
hrv_score = predictions[1][0][0] * 100
rhr_score = predictions[2][0][0] * 100
recovery_score = predictions[3][0][0] * 100
readiness_score = predictions[4][0][0] * 100

print(f"Sleep Score: {sleep_score:.1f}/100")
print(f"HRV Score: {hrv_score:.1f}/100")
print(f"RHR Score: {rhr_score:.1f}/100")
print(f"Recovery Score: {recovery_score:.1f}/100")
print(f"Readiness Score: {readiness_score:.1f}/100")
'''
    
    print("\nModel telah berhasil disimpan!")
    print("File yang dihasilkan:")
    print("1. fitbit_full_model.keras - Model utama")
    print("2. fitbit_full_model.h5    - Model (format lama)")
    print("3. model_metadata.json     - Informasi model")
    print("4. fitbit_encoder2.keras   - Encoder saja (sudah ada)")
    
    # Simpan contoh kode ke file
    with open('example_usage.py', 'w') as f:
        f.write(example_code)
    print("\n✅ Contoh kode penggunaan disimpan: 'example_usage.py'")